In [239]:
from keras.models import Model
from keras.layers.recurrent import LSTM
from keras.layers import Dense, Input, Embedding
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import ModelCheckpoint
from keras.utils.vis_utils import plot_model
from keras.preprocessing.text import Tokenizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import TweetTokenizer
from nltk.stem.wordnet import WordNetLemmatizer
from wordcloud import WordCloud
import gensim
from gensim.models import word2vec
import logging
import  tensorflow as tf
from collections import Counter
import nltk
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import urllib.request
import os
import sys
import zipfile
import logging
import pydot
import graphviz
import re
import re
from functools import reduce
from collections import defaultdict
import collections
import nltk
from nltk.corpus import brown
import wordsegment
from wordsegment import load, segment
from operator import itemgetter
import keras

In [4]:
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

logging.basicConfig(level=logging.DEBUG)

In [5]:
data = pd.read_csv("train.csv")

In [6]:
data = data.fillna(" ")

In [7]:
with open('vocabulary.txt', 'r') as f:
    s = f.read()

In [8]:
s = s.strip().split("\n")

In [9]:
vocab = [s0.strip() for s0 in s]

In [10]:
vocab

['!',
 "'",
 ',',
 '.',
 '?',
 'a',
 'aa',
 'aaa',
 'aaaa',
 'aaaaaaaaaaaaaaaahhhhhhhh',
 'aaaaaaaaaaaahhhhhhhhhhhh',
 'aaron',
 'ab',
 'aba',
 'abandon',
 'abandoned',
 'abandoning',
 'abandonment',
 'abating',
 'abb',
 'abba',
 'abbreviated',
 'abbs',
 'abc',
 'abcsports',
 'abd',
 'abel',
 'abernathy',
 'abg',
 'abhijeet',
 'abides',
 'abigail',
 'abilities',
 'ability',
 'abitibi',
 'abl',
 'able',
 'ablity',
 'ably',
 'abn',
 'abnormality',
 'abo',
 'aboard',
 'about',
 'above',
 'abrade',
 'abramo',
 'abrams',
 'abreast',
 'abri',
 'abroad',
 'abruptly',
 'abs',
 'absence',
 'absences',
 'absense',
 'absent',
 'abshire',
 'absolute',
 'absolutely',
 'absolutley',
 'absolve',
 'absorb',
 'absorbed',
 'absorbs',
 'abstraction',
 'absurd',
 'absurdity',
 'abt',
 'abu',
 'abundantly',
 'abuse',
 'abused',
 'abx',
 'ac',
 'aca',
 'acacia',
 'academic',
 'academy',
 'acc',
 'acce',
 'accelerate',
 'accelerated',
 'accelerating',
 'acceleration',
 'accellera',
 'accent',
 'accenture',
 

In [191]:
def tokenize(sent):
    '''Return the tokens of a sentence including punctuation.
    >>> tokenize('Bob dropped the apple. Where is the apple?')
    ['Bob', 'dropped', 'the', 'apple', '.', 'Where', 'is', 'the', 'apple', '?']
    '''
    return [x.strip() for x in re.findall(r"[\w']+|[.,!?]", sent) if x.strip()]

In [12]:
def get_data(data):
    questions = data.Context.apply(tokenize).values
    answers = data.Utterance.apply(tokenize).values

    #questions.apply(tokenize)
    #answers.apply(tokenize)

    data = [(q, a) for q, a in zip(questions, answers)]
    return data

In [246]:
train = get_data(data)

In [14]:
len(vocab)

21234

In [163]:
data.head()

,Context,Utterance
0,i'm good why aren't you online in instant mess...,what are you doing for lunch ?
1,yep back in the saddle again for two weeks tha...,are you in today ?
2,goss and you can't go now because of some lag ...,what happened to the gambling trip ?
3,credit is not throwing you off . they did send...,marie can you please check on this and let ric...
4,"limor , the instructions below should help you...",put this in a safe place !


In [243]:
input1 = data['Context'].fillna("")
output1 = data['Utterance'].fillna("")

In [247]:
train = list(map(pre_process,train))

In [248]:
target_counter = Counter()
for question,answer in train:
    for w in answer:
        target_counter[w] += 1

In [249]:
target_word2idx = dict()  

In [250]:
for idx, word in enumerate(target_counter.most_common(len(vocab))):
    target_word2idx[word[0]] = idx + 1

In [251]:
if 'UNK' not in target_word2idx:
    target_word2idx['UNK'] = 0

In [252]:
target_idx2word = dict([(idx, word) for word, idx in target_word2idx.items()])

In [253]:
GLOVE_EMBEDDING_SIZE = 100
HIDDEN_UNITS = 64
BATCH_SIZE = 64
NUM_EPOCHS = 3

In [254]:
input_texts = [question for question,answer in train]
target_texts = [answer for question,answer in train]

In [255]:
sentences = input_texts
model = word2vec.Word2Vec(sentences)
vocab_size = len(model.wv.vocab)
print("Vocab size :: ",vocab_size)
print(model.wv.index2word[0:10])
print(model.wv.index2word[vocab_size - 1:vocab_size-10])
word2em = np.zeros((len(model.wv.vocab), 100))
for i in range(len(model.wv.vocab)):
    embedding_vector = model.wv[model.wv.index2word[i]]
    if embedding_vector is not None:
        word2em[i] = embedding_vector

INFO:gensim.models.word2vec:collecting all words and their counts
INFO:gensim.models.word2vec:PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO:gensim.models.word2vec:PROGRESS: at sentence #10000, processed 478755 words, keeping 15117 word types
INFO:gensim.models.word2vec:collected 17243 word types from a corpus of 733117 raw words and 15284 sentences
INFO:gensim.models.word2vec:Loading a fresh vocabulary
INFO:gensim.models.word2vec:effective_min_count=5 retains 7668 unique words (44% of original 17243, drops 9575)
INFO:gensim.models.word2vec:effective_min_count=5 leaves 714267 word corpus (97% of original 733117, drops 18850)
INFO:gensim.models.word2vec:deleting the raw counts dictionary of 17243 items
INFO:gensim.models.word2vec:sample=0.001 downsamples 52 most-common words
INFO:gensim.models.word2vec:downsampling leaves estimated 535756 word corpus (75.0% of prior 714267)
INFO:gensim.models.base_any2vec:estimated required memory for 7668 words and 100 dimension

Vocab size ::  7668
['the', 'to', 'i', 'and', 'you', 'a', 'of', 'is', 'in', 'for']
[]


In [256]:
len(encoder_input_wids)

37

In [257]:
num_decoder_tokens = len(target_idx2word)+1
input_texts_word2em = []
encoder_max_seq_length = 0
decoder_max_seq_length = 0
for input_words, target_words in train:
    encoder_input_wids = []
    for w in input_words:
        emb = np.zeros(shape=GLOVE_EMBEDDING_SIZE)
        if w in word2em:
            emb = word2em[w]
        encoder_input_wids.append(emb)
    input_texts_word2em.append(encoder_input_wids)
    encoder_max_seq_length = max(len(encoder_input_wids), encoder_max_seq_length)
    decoder_max_seq_length = max(len(target_words), decoder_max_seq_length)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if __name__ == '__main__':


In [258]:
context = dict()
context['num_decoder_tokens'] = num_decoder_tokens
context['encoder_max_seq_length'] = encoder_max_seq_length
context['decoder_max_seq_length'] = decoder_max_seq_length

In [259]:
max_encoder_seq_length = context['encoder_max_seq_length']
max_decoder_seq_length = context['decoder_max_seq_length']
num_decoder_tokens = context['num_decoder_tokens']

In [260]:
print("context: ",context)

context:  {'num_decoder_tokens': 8460, 'encoder_max_seq_length': 1154, 'decoder_max_seq_length': 30}


In [344]:
encoder_inputs = Input(shape=(None, GLOVE_EMBEDDING_SIZE), name='encoder_inputs')
encoder_lstm1 = LSTM(units=HIDDEN_UNITS, return_state=True, name="encoder_lstm1" , dropout=0.2)

In [345]:
encoder_outputs, encoder_state_h, encoder_state_c = encoder_lstm1(encoder_inputs)

In [346]:
encoder_states = [encoder_state_h, encoder_state_c]

In [347]:
decoder_inputs = Input(shape=(None, GLOVE_EMBEDDING_SIZE), name='decoder_inputs')
decoder_lstm = LSTM(units=HIDDEN_UNITS, return_sequences=True, return_state=True, name='decoder_lstm', dropout=0.2)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax', name='decoder_dense')
decoder_outputs = decoder_dense(decoder_outputs)

In [348]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [349]:
 print(model.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_inputs (InputLayer)     (None, None, 100)    0                                            
__________________________________________________________________________________________________
decoder_inputs (InputLayer)     (None, None, 100)    0                                            
__________________________________________________________________________________________________
encoder_lstm1 (LSTM)            [(None, 64), (None,  42240       encoder_inputs[0][0]             
__________________________________________________________________________________________________
decoder_lstm (LSTM)             [(None, None, 64), ( 42240       decoder_inputs[0][0]             
                                                                 encoder_lstm1[0][1]              
          

In [350]:
import keras.backend as K

def precision_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

In [355]:
optimizer = keras.optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.0)
model.compile(optimizer=optimizer, loss='categorical_crossentropy',
              metrics=['accuracy', precision_m])

In [356]:
Xtrain, Xtest, Ytrain, Ytest = train_test_split(input_texts_word2em, target_texts, test_size=0.2, random_state=42)

In [357]:
print("Length of train data:: ", len(Xtrain))
print("Length of test data:: ", len(Xtest))

Length of train data::  12227
Length of test data::  3057


In [358]:
def generate_batch(input_word2em_data, output_text_data):
    
    num_batches = len(input_word2em_data) // BATCH_SIZE
    print("context:: \n", context)
    print("len of input data :: ", len(input_word2em_data))
    print("num of batches :: ", num_batches)
    
    while True:
        for batchIdx in range(0, num_batches):
            
            start = batchIdx * BATCH_SIZE
            end = (batchIdx + 1) * BATCH_SIZE
            
            encoder_input_data_batch = pad_sequences(input_word2em_data[start:end], context['encoder_max_seq_length'])
            decoder_target_data_batch = np.zeros(shape=(BATCH_SIZE, context['decoder_max_seq_length'], num_decoder_tokens))
            decoder_input_data_batch = np.zeros(shape=(BATCH_SIZE, context['decoder_max_seq_length'], GLOVE_EMBEDDING_SIZE))
            
            for lineIdx, target_words in enumerate(output_text_data[start:end]):
                for idx, w in enumerate(target_words):
                    w2idx = target_word2idx['UNK']  # default UNK
                    if w in target_word2idx:
                        w2idx = target_word2idx[w]
                    if w in word2em:
                        decoder_input_data_batch[lineIdx, idx, :] = word2em[w]
                    if idx > 0:
                        decoder_target_data_batch[lineIdx, idx - 1, w2idx] = 1
            yield [encoder_input_data_batch, decoder_input_data_batch], decoder_target_data_batch

In [359]:
train_gen = generate_batch(Xtrain, Ytrain)
test_gen = generate_batch(Xtest, Ytest)

In [360]:
train_num_batches = len(Xtrain) // BATCH_SIZE
test_num_batches = len(Xtest) // BATCH_SIZE

In [361]:
model.fit_generator(generator=train_gen, steps_per_epoch=train_num_batches,
                    epochs=5,
                    verbose=1, validation_data=test_gen, validation_steps=test_num_batches )

Epoch 1/5
context:: 
 {'num_decoder_tokens': 8460, 'encoder_max_seq_length': 1154, 'decoder_max_seq_length': 30}
len of input data ::  3057
num of batches ::  47
context:: 
 {'num_decoder_tokens': 8460, 'encoder_max_seq_length': 1154, 'decoder_max_seq_length': 30}
len of input data ::  12227
num of batches ::  191


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:23: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison


191/191 [==============================] - 1194s 6s/step - loss: 2.7230 - acc: 0.0176 - precision_m: 0.0000e+00 - val_loss: 2.6244 - val_acc: 0.0163 - val_precision_m: 0.0000e+00
Epoch 2/5
102/191 [===============>..............] - ETA: 9:19 - loss: 2.6122 - acc: 0.0152 - precision_m: 0.0000e+00

KeyboardInterrupt: 

In [ ]:
model.save_weights(WEIGHT_FILE_PATH)

In [223]:
encoder_model = Model(encoder_inputs, encoder_states)

In [224]:
decoder_state_inputs = [Input(shape=(HIDDEN_UNITS,)), Input(shape=(HIDDEN_UNITS,))]
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_state_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model([decoder_inputs] + decoder_state_inputs, [decoder_outputs] + decoder_states)

In [225]:
def pre_process(qa):
    
    question,answer = qa
    w_q = [w for w in question if (len(w)!=1)|(w.isalpha())]
    
    w_a = [w for w in answer if (len(w)!=1)|(w.isalpha())]
    return (w_q,w_a)

In [228]:
def reply(input_text):
    input_seq = []
    input_emb = []
    print("input text:: \n\n ", input_text)
    # pre-processing only the input data 
    clean_input_text = input_text
    for word in nltk.word_tokenize(clean_input_text.lower()):
        emb = np.zeros(shape=GLOVE_EMBEDDING_SIZE)
        if word in word2em:
            emb = word2em[word]
        input_emb.append(emb)
    input_seq.append(input_emb)
    input_seq = pad_sequences(input_seq, max_encoder_seq_length)
    states_value = encoder_model.predict(input_seq)
    target_seq = np.zeros((1, 1, GLOVE_EMBEDDING_SIZE))
    target_text = ''
    target_text_len = 0
    terminated = False
    while not terminated:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        print("output tokens shape  :: \n\n ", output_tokens.shape)
        sample_token_idx = np.argmax(output_tokens[0, -1, :])
        sample_word = target_idx2word[sample_token_idx]
        target_text_len += 1
        if sample_word != 'start' and sample_word != 'end':
            print("sample word :: ", sample_word)
            target_text += ' ' + sample_word
        if sample_word == 'end' or target_text_len >= max_decoder_seq_length:
            terminated = True
        target_seq = np.zeros((1, 1, GLOVE_EMBEDDING_SIZE))
        if sample_word in word2em:
            target_seq[0, 0, :] = word2em[sample_word]
        states_value = [h, c]
    return target_text.strip()

In [236]:
print(reply('what are you doing for lunch'))

input text:: 

  what are you doing for lunch
output tokens shape  :: 

  (1, 1, 8460)
sample word ::  you
output tokens shape  :: 

  (1, 1, 8460)
sample word ::  you
output tokens shape  :: 

  (1, 1, 8460)
sample word ::  the
output tokens shape  :: 

  (1, 1, 8460)
sample word ::  to
output tokens shape  :: 

  (1, 1, 8460)
sample word ::  the
output tokens shape  :: 

  (1, 1, 8460)
sample word ::  the
output tokens shape  :: 

  (1, 1, 8460)
sample word ::  the
output tokens shape  :: 

  (1, 1, 8460)
sample word ::  the
output tokens shape  :: 

  (1, 1, 8460)
sample word ::  the
output tokens shape  :: 

  (1, 1, 8460)
sample word ::  to
output tokens shape  :: 

  (1, 1, 8460)
sample word ::  to
output tokens shape  :: 

  (1, 1, 8460)
sample word ::  the
output tokens shape  :: 

  (1, 1, 8460)
sample word ::  the
output tokens shape  :: 

  (1, 1, 8460)
sample word ::  the
output tokens shape  :: 

  (1, 1, 8460)
sample word ::  the
output tokens shape  :: 

  (1, 1, 8460)
s

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if __name__ == '__main__':
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:31: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison


In [149]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/rachelchen/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [155]:
eng_stopwords = set(stopwords.words("english"))

In [154]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/rachelchen/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [159]:
APPO = {
"aren't" : "are not",
"can't" : "cannot",
"couldn't" : "could not",
"didn't" : "did not",
"doesn't" : "does not",
"don't" : "do not",
"hadn't" : "had not",
"hasn't" : "has not",
"haven't" : "have not",
"he'd" : "he would",
"he'll" : "he will",
"he's" : "he is",
"i'd" : "I would",
"i'd" : "I had",
"i'll" : "I will",
"i'm" : "I am",
"isn't" : "is not",
"it's" : "it is",
"it'll":"it will",
"i've" : "I have",
"let's" : "let us",
"mightn't" : "might not",
"mustn't" : "must not",
"shan't" : "shall not",
"she'd" : "she would",
"she'll" : "she will",
"she's" : "she is",
"shouldn't" : "should not",
"that's" : "that is",
"there's" : "there is",
"they'd" : "they would",
"they'll" : "they will",
"they're" : "they are",
"they've" : "they have",
"we'd" : "we would",
"we're" : "we are",
"weren't" : "were not",
"we've" : "we have",
"what'll" : "what will",
"what're" : "what are",
"what's" : "what is",
"what've" : "what have",
"where's" : "where is",
"who'd" : "who would",
"who'll" : "who will",
"who're" : "who are",
"who's" : "who is",
"who've" : "who have",
"won't" : "will not",
"wouldn't" : "would not",
"you'd" : "you would",
"you'll" : "you will",
"you're" : "you are",
"you've" : "you have",
"'re": " are",
"wasn't": "was not",
"we'll":" will",
"didn't": "did not",
    }

In [160]:
lem = WordNetLemmatizer()

In [ ]:
def evaluate_recall(y, y_test, k=1):
    num_examples = float(len(y))
    num_correct = 0
    for predictions, label in zip(y, y_test):
        if label in predictions[:k]:
            num_correct += 1
    return num_correct/num_examples


def test_precision_at_k(pred_opt, feed_dict, k, sess):
    sims = pred_opt.eval(session=sess, feed_dict=feed_dict)
    labels = range(0, len(sims))
    for i, pred_vector in enumerate(sims):
        sims[i] = [i[0] for i in sorted(enumerate(pred_vector), key=lambda x: x[1])][::-1]

    recall_score = evaluate_recall(sims, labels, k)
    return recall_score